In [ ]:
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import numpy as np
from langchain_ollama import OllamaLLM
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef
from tqdm import tqdm

llm = OllamaLLM(model="llama_detector1")

In [ ]:
test_df = pd.read_csv("test_10k.csv")

In [ ]:
# Funkcia na predikciu pre jeden text
def predict_single(text):
    # Volanie LLM API (predpokladáme, že vráti 0 alebo 1)
    return llm.invoke(text)

# Paralelné spracovanie pomocou vlákien
def parallel_prediction(data):
    predictions = []
    with ThreadPoolExecutor(max_workers=8) as executor:  # Nastavenie počtu vlákien
        # Použitie tqdm pre progress bar
        for result in tqdm(executor.map(predict_single, data), total=len(data), desc="Processing Predictions"):
            predictions.append(result)
    return predictions

if __name__ == '__main__':
    # Načítanie dát
    test_df = pd.read_csv("test_10k.csv")

    # Paralelné predikcie s progress barom
    test_df['predicted'] = parallel_prediction(test_df['data'].tolist())
    
    # Overenie výsledku
    print(test_df)
    test_df.to_csv('test_pred.csv', index=False)

In [ ]:
test_df = pd.read_csv("test_pred.csv")
test_df["label"] = test_df["label"].astype("int64")
test_df["predicted"] = test_df["predicted"].astype("int64")
test_df.to_csv('test_pred.csv', index=False)